In [3]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.
import html5lib

import numpy as np
import pandas as pd

import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains

In [8]:
regularForHref = r'(href=\"([^"]*)\")'
regularExpForHref = re.compile(regularForHref)
hrefArray = []
titleArray = []

In [21]:
texts = []
hrefs = []
titles = []

In [15]:
themes = ['military', 'economy', 'columnist', 'dk']
indexes = [0, 1, 2, 3]
themeIndexes = dict(zip(indexes, themes))
themeIndexes

{0: 'military', 1: 'economy', 2: 'columnist', 3: 'dk'}

In [12]:
driver = webdriver.Chrome()

In [22]:
for theme in themes:
  driver.get(f"https://profile.ru/{theme}/")
  count = 0
  while count < 50:
    ActionChains(driver).scroll_by_amount(delta_x=0, delta_y=9999).perform()
    time.sleep(5)
    count+=1
    
  articles = driver.find_elements(By.CLASS_NAME, 'article__title')
  
  for article in articles:
    titles.append(article.text)
    hrefs.append(article.get_attribute('href'))
    

In [74]:
# Здесь нужен цикл
pageNum = 1
while pageNum < 52:
  url = f"https://cdn.profile.ru/wp-content/uploads/alm-cache/135928473761329073/page-{pageNum}.html"
  res = requests.get(url)
  textRes = res.text
  textRes
  res.encoding
  bsTitles = BeautifulSoup(res.content, "html.parser")
  bsTitles.get_text()

  bs = BeautifulSoup(res.content)

  hrefTitles = bs.find_all(name='h2', attrs={'class':'newslist__title'})
  titles = [l.find('a').text for l in hrefTitles]
  
  for line in titles:
    titleArray.append(line)

  for line in hrefTitles:
    hrefArray.append(re.search(regularExpForHref, str (line)).group(2))
      
  pageNum += 1
  res.close()


In [76]:
hrefArray

['https://profile.ru/news/politics/v-mid-yaponii-zayavili-chto-strana-predostavila-ukraine-pomoshh-na-7-6-milliarda-1329021/',
 'https://profile.ru/news/accidents/belgorodskaya-oblast-podverglas-novoj-atake-bespilotnikov-1329025/',
 'https://profile.ru/news/society/klimatolog-rasskazal-o-sezone-uraganov-v-moskve-1329015/',
 'https://profile.ru/news/economy/rossijskij-holding-segezha-group-prodal-evropejskie-aktivy-za-e1-1329003/',
 'https://profile.ru/news/society/zhiteli-moskvy-mogut-vybrat-cveta-kotorymi-oboznachat-novye-linii-metro-1329001/',
 'https://profile.ru/news/economy/cena-na-zoloto-vernulas-k-rostu-posle-nebolshogo-snizheniya-1328996/',
 'https://profile.ru/news/politics/reuters-premer-ministr-yaponii-iskljuchil-vstuplenie-strany-v-nato-1328995/',
 'https://profile.ru/news/economy/kitajskij-juan-opustilsya-k-dollaru-pochti-do-minimuma-za-shest-mesyacev-1328983/',
 'https://profile.ru/news/politics/po-itogam-peregovorov-rossiya-i-kitaj-podpisali-5-dokumentov-1328984/',
 'htt

In [117]:
titleArrayPD = pd.Series(titleArray)
hrefArrayPD = pd.Series(hrefArray)

concatArray = pd.DataFrame(columns=['href','title','text','comments'])
concatArray['href'] = hrefArrayPD
concatArray['title'] = titleArrayPD
concatArray.to_csv('hrefs.CSV', index=False)
concatArray

,href,title,text,comments
0,https://profile.ru/news/politics/v-mid-yaponii...,"В МИД Японии заявили, что страна предоставила ...",NaN,NaN
1,https://profile.ru/news/accidents/belgorodskay...,Белгородская область подверглась новой атаке б...,NaN,NaN
2,https://profile.ru/news/society/klimatolog-ras...,Климатолог рассказал о сезоне ураганов в Москве,NaN,NaN
3,https://profile.ru/news/economy/rossijskij-hol...,Российский холдинг Segezha Group продал европе...,NaN,NaN
4,https://profile.ru/news/society/zhiteli-moskvy...,"Жители Москвы могут выбрать цвета, которыми об...",NaN,NaN
...,...,...,...,...
403,https://profile.ru/news/politics/lidery-g7-dog...,Лидеры G7 договорились продолжить поддержку Ук...,NaN,NaN
404,https://profile.ru/news/economy/v-kabmine-ozhi...,"В кабмине ожидают, что рынок БПЛА в России по ...",NaN,NaN
405,https://profile.ru/news/cars/ria-novosti-v-ros...,РИА Новости: в России средний размер автокреди...,NaN,NaN
406,https://profile.ru/news/accidents/vsu-obstrely...,ВСУ обстреляли село Муром Белгородской области,NaN,NaN


In [118]:
textArray = []

In [120]:
for line in hrefArrayPD:
  resNews = requests.get(line)
  resNewsText = BeautifulSoup(resNews.content, 'html.parser')
  textRaw = resNewsText.find_all("div", attrs={'class':'micromarking'})
  text = " ".join([p.text for p in textRaw])
  textArray.append(text)
  resNews.close()
  

In [121]:
textArrayPD = pd.DataFrame(textArray)
textArrayPD

,0
0,\nВласти Японии к настоящему времени предостав...
1,"\nНочью в среду, 24 мая, в Белгородской област..."
2,\nВ столичном регионе в конце мая – начале июн...
3,\nРоссийский лесопромышленный холдинг Segezha ...
4,\nЖители Москвы могут проголосовать за одну из...
...,...
412,"\nЛидеры стран ""Большой семерки"" (G7) за день ..."
413,\nОбъем рынка беспилотных летательных аппарато...
414,\nСредний размер кредита на покупку автомобиля...
415,"\nУтром в субботу, 20 мая, под обстрел со стор..."


In [125]:
concatArray['text'] = textArrayPD
concatArray.to_csv('hrefs.CSV', index=False)
concatArray

,href,title,text,comments
0,https://profile.ru/news/politics/v-mid-yaponii...,"В МИД Японии заявили, что страна предоставила ...",\nВласти Японии к настоящему времени предостав...,NaN
1,https://profile.ru/news/accidents/belgorodskay...,Белгородская область подверглась новой атаке б...,"\nНочью в среду, 24 мая, в Белгородской област...",NaN
2,https://profile.ru/news/society/klimatolog-ras...,Климатолог рассказал о сезоне ураганов в Москве,\nВ столичном регионе в конце мая – начале июн...,NaN
3,https://profile.ru/news/economy/rossijskij-hol...,Российский холдинг Segezha Group продал европе...,\nРоссийский лесопромышленный холдинг Segezha ...,NaN
4,https://profile.ru/news/society/zhiteli-moskvy...,"Жители Москвы могут выбрать цвета, которыми об...",\nЖители Москвы могут проголосовать за одну из...,NaN
...,...,...,...,...
403,https://profile.ru/news/politics/lidery-g7-dog...,Лидеры G7 договорились продолжить поддержку Ук...,\nПравительство России расширило доступ произв...,NaN
404,https://profile.ru/news/economy/v-kabmine-ozhi...,"В кабмине ожидают, что рынок БПЛА в России по ...","\nЛидеры государств, входящих в ""Большую семер...",NaN
405,https://profile.ru/news/cars/ria-novosti-v-ros...,РИА Новости: в России средний размер автокреди...,\nNikkei: Япония запретит предоставлять России...,NaN
406,https://profile.ru/news/accidents/vsu-obstrely...,ВСУ обстреляли село Муром Белгородской области,\nИтальянский ракетный эскадренный миноносец п...,NaN
